#Importing libraries

In [0]:
import pandas as pd
import pyspark
from pyspark.sql.functions import lit
from pyspark.sql.types import DateType, IntegerType, StringType, DoubleType
from datetime import datetime


#Databases

In [0]:
## Inserir yours databases
lista_database = [''] 

# Report data generation

In [0]:
#Declaration of dataframes
df_detalhes = 0
df_detalhes_appended = pd.DataFrame()
data_atual = datetime.now().strftime("%d/%m/%Y")

#Start of the Loop to see the databases
for db in lista_database_talend:
  df_metadata = spark.sql('SHOW TABLES IN  '+db)
  df_pandas_md  = df_metadata.select('database', 'tableName').toPandas()

#list of Schemas and tables inside pandas dataframe
  for database,table in df_pandas_md.values:
    if not isinstance(df_detalhes, pyspark.sql.dataframe.DataFrame):
      try:  #First pass to each database  
        df_detalhes = spark.sql(' DESCRIBE DETAIL '+database+'.'+table)
        print('1º Ok /', database+'.'+table, ' / criação')
      except:
         print('1º Error', database+'.'+table)
    else:
      try:  #Second pass for each database  
        df_detalhes = df_detalhes.union(spark.sql(' DESCRIBE DETAIL '+database+'.'+table))
        print('2º Ok /', database+'.'+table, ' / union')
      except:
        print('2º Error', database, table)
df_detalhes = df_detalhes.withColumn("dataExec", lit(data_atual).cast(StringType()))
df_detalhes = df_detalhes.toPandas()
df_detalhes_appended = df_detalhes_appended.append(df_detalhes)
  
df_final = spark.createDataFrame(df_detalhes_appended[['name','createdAt','lastModified', 'sizeInBytes', 'dataExec']].sort_values('lastModified', ascending=False)).toPandas()
print("End of loads.")
df_final.head()

#Table creation

In [0]:
#Creates the dataframe with the final result of data generation
#create a temporary table to do the comparison
df_final = spark.createDataFrame(df_final)
df_final.createOrReplaceTempView("ctrl_carga_temp")

##Comparison

In [0]:
%sql
--Create a failure table based on the data that was inserted into the failed temp table
create or replace table stage.ctrl_carga_falhas as
  select org.* 
  from ctrl_carga_temp tmp
  inner join stage.ctrl_carga_falhas org
    on org.name = tmp.name 
  where org.lastModified >= tmp.lastModified
  and org.sizeInBytes >= tmp.sizeInBytes	

In [0]:
#inserts the data collected by the report into the physical table
permanent_table_name = "stage.ctrl_carga_falhas"
df_final.write.mode('append').format("delta").saveAsTable(permanent_table_name)

In [0]:
%sql
--Delete data from table with execution date less than current day - 2
DELETE FROM stage.ctrl_carga_falhas
WHERE date_format(to_date(dataExec, 'dd/MM/yyyy'), 'yyyy-MM-dd') <= date_sub(current_date, 2)

In [0]:
%sql
--Delete data from table with execution date less than current day - 1
DELETE FROM stage.ctrl_carga
WHERE date_format(to_date(dataExec, 'dd/MM/yyyy'), 'yyyy-MM-dd') <= date_sub(current_date, 1)

##Report extraction

In [0]:
#crash report
df_falha = spark.read.table('stage.ctrl_carga_falhas')
df_falha.display()

In [0]:
#data loads report
df_table = spark.read.table('stage.ctrl_carga')
df_table.display()